In [ ]:
# Libraries

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import joblib
import os # setting working directory
import sklearn as sk
from tabulate import tabulate
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import f1_score
from statistics import harmonic_mean
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Read in the data from file
# train = pd.read_csv('../input/reddit-data/reddit_training.csv')
train = pd.read_csv('../input/dataset/reddit_training.csv')
# train = train.drop('Controls', axis=1)
# train = train.drop('Graphics', axis=1)
# train = train.drop('Story', axis=1)
# train = train.drop('Bugs', axis=1)

# print out the first few lines
#train.head()

In [ ]:
# Read in the data from file
# test = pd.read_csv('../input/reddit-data/reddit_testing.csv')
test = pd.read_csv('../input/dataset/reddit_testing.csv')
# test = test.drop('Controls', axis=1)
# test = test.drop('Graphics', axis=1)
# test = test.drop('Story', axis=1)
# test = test.drop('Bugs', axis=1)

In [ ]:
# preprocess the data
def preprocess_data(data):
    # Convert text to lowercase and replace new lines with spaces
    data['Overall Sentiment'] = data['Overall Sentiment'].str.strip().str.lower()
    data['Controls'] = data['Controls'].str.strip().str.lower()
    data['Graphics'] = data['Graphics'].str.strip().str.lower()
    data['Story'] = data['Story'].str.strip().str.lower()
    data['Bugs'] = data['Bugs'].str.strip().str.lower()
    data['Comment Body'] = data['Comment Body'].str.strip().str.lower()
    data['Comment Body'] = data['Comment Body'].str.replace("\n", " ")
    return data


# call preprocess function
test = preprocess_data(test)
train = preprocess_data(train)

# define train and test sets for Overall Sentiment
train_overall = train[train['Overall Sentiment'] != 'neutral']
test_overall = test[test['Overall Sentiment'] != 'neutral']

# define train and test sets for Control Sentiment
train_controls = train[train['Controls'] != 'neutral']
test_controls = test[test['Controls'] != 'neutral']

# define train and test sets for Story Sentiment
train_story = train[train['Story'] != 'neutral']
test_story = test[test['Story'] != 'neutral']

# define train and test sets for Control Sentiment
train_graphics = train[train['Graphics'] != 'neutral']
test_graphics = test[test['Graphics'] != 'neutral']

# define train and test sets for Control Sentiment
train_bugs = train[train['Bugs'] != 'neutral']
test_bugs = test[test['Bugs'] != 'neutral']

train_overall.head()

In [ ]:
##### gets n-grams
# chooses n's for n-grams
min_n = 1
max_n = 4

# sets ups n-gram calculator
# character partition
# ngram_calculator = CountVectorizer(ngram_range = (min_n, max_n), analyzer = 'char')
# word partition
ngram_calculator = CountVectorizer(ngram_range = (min_n, max_n), analyzer = 'word')
ngram_calculator.fit(train_overall['Comment Body'])
# gets ngram (count) features
x_train_overall = ngram_calculator.transform(train_overall['Comment Body'])
x_test_overall = ngram_calculator.transform(test_overall['Comment Body'])

# redo for other facets
# controls
#ngram_calculator = CountVectorizer(ngram_range = (min_n, max_n), analyzer = 'word')
ngram_calculator.fit(train_controls['Comment Body'])
x_train_controls = ngram_calculator.transform(train_controls['Comment Body'])
x_test_controls = ngram_calculator.transform(test_controls['Comment Body'])
# graphics
#ngram_calculator = CountVectorizer(ngram_range = (min_n, max_n), analyzer = 'word')
ngram_calculator.fit(train_graphics['Comment Body'])
x_train_graphics = ngram_calculator.transform(train_graphics['Comment Body'])
x_test_graphics = ngram_calculator.transform(test_graphics['Comment Body'])
# story
#ngram_calculator = CountVectorizer(ngram_range = (min_n, max_n), analyzer = 'word')
ngram_calculator.fit(train_story['Comment Body'])
x_train_story = ngram_calculator.transform(train_story['Comment Body'])
x_test_story = ngram_calculator.transform(test_story['Comment Body'])
# bugs
#ngram_calculator = CountVectorizer(ngram_range = (min_n, max_n), analyzer = 'word')
ngram_calculator.fit(train_bugs['Comment Body'])
x_train_bugs = ngram_calculator.transform(train_bugs['Comment Body'])
x_test_bugs = ngram_calculator.transform(test_bugs['Comment Body'])

In [ ]:
# Testin printings
# print(train_bugs)

In [ ]:
##### gets labels for overall sentiment
y_train_overall = train_overall['Overall Sentiment']
y_test_overall = test_overall['Overall Sentiment']

##### gets labels for controls sentiment
y_train_controls = train_controls['Controls']
y_test_controls = test_controls['Controls']

##### gets labels for graphics sentiment
y_train_graphics = train_graphics['Graphics']
y_test_graphics = test_graphics['Graphics']

##### gets labels for story sentiment
y_train_story = train_story['Story']
y_test_story = test_story['Story']

##### gets labels for bugs sentiment
y_train_bugs = train_bugs['Bugs']
y_test_bugs = test_bugs['Bugs']
    
# remove neutral reviews
# y_train = y_train[y_train != 'neutral']
# y_test = y_test[y_test != 'neutral']
    
# converts Positive/Negative to binary for sci-kit
review2bin = {'negative':0, 'positive':1}

y_train_overall = [review2bin[el] for el in y_train_overall]
y_test_overall = [review2bin[el] for el in y_test_overall]

y_train_controls = [review2bin[el] for el in y_train_controls]
y_test_controls = [review2bin[el] for el in y_test_controls]

y_train_graphics = [review2bin[el] for el in y_train_graphics]
y_test_graphics = [review2bin[el] for el in y_test_graphics]

y_train_story = [review2bin[el] for el in y_train_story]
y_test_story = [review2bin[el] for el in y_test_story]

y_train_bugs = [review2bin[el] for el in y_train_bugs]
y_test_bugs = [review2bin[el] for el in y_test_bugs]

In [ ]:
##################### splits training data sets into train/valid ##############################
x_train_train_overall, x_train_valid_overall, y_train_train_overall, y_train_valid_overall = train_test_split(x_train_overall, y_train_overall, test_size = 0.20, random_state = 0)
################## splits facets training data sets into train/valid ##########################
x_train_train_controls, x_train_valid_controls, y_train_train_controls, y_train_valid_controls = train_test_split(x_train_controls, y_train_controls, test_size = 0.20, random_state = 0)
x_train_train_graphics, x_train_valid_graphics, y_train_train_graphics, y_train_valid_graphics = train_test_split(x_train_graphics, y_train_graphics, test_size = 0.20, random_state = 0)
x_train_train_story, x_train_valid_story, y_train_train_story, y_train_valid_story = train_test_split(x_train_story, y_train_story, test_size = 0.20, random_state = 0)
x_train_train_bugs, x_train_valid_bugs, y_train_train_bugs, y_train_valid_bugs = train_test_split(x_train_bugs, y_train_bugs, test_size = 0.20, random_state = 0)

In [ ]:
# function that fits regression model and assesses performance
def logistic_wrapper(x_fit, y_fit, x_performance, y_performance, alpha):
    # runs logistic regression
    model = LogisticRegression(C = 1/alpha, solver = 'liblinear')
    model.fit(x_fit, y_fit)
    
    # calculates performance (precision, recall, F1)
    labels = y_performance
    predictions = model.predict(x_performance)
    precision, recall, f1, support = precision_recall_fscore_support(labels, predictions, average ='binary')
        
    return f1, precision, recall, predictions

In [ ]:
# This is where the table needs data from for analysis
##### fits a regression model and assesses performance
f1, precision, recall, predictions_overall = logistic_wrapper(x_train_overall, y_train_overall, x_test_overall, y_test_overall, 0.1)
    
# print results
print('F1: ', f1)
print('precision: ', precision)
print('recall: ', recall)

In [ ]:
# This is where the table needs data from for analysis
##### fits a regression model and assesses performance
f1, precision, recall, predictions_controls = logistic_wrapper(x_train_controls, y_train_controls, x_test_controls, y_test_controls, 0.1)
    
# print results
print('F1: ', f1)
print('precision: ', precision)
print('recall: ', recall)

In [ ]:
# This is where the table needs data from for analysis
##### fits a regression model and assesses performance
f1, precision, recall, predictions_graphics = logistic_wrapper(x_train_graphics, y_train_graphics, x_test_graphics, y_test_graphics, 0.1)
    
# print results
print('F1: ', f1)
print('precision: ', precision)
print('recall: ', recall)

In [ ]:
# This is where the table needs data from for analysis
##### fits a regression model and assesses performance
f1, precision, recall, predictions_story = logistic_wrapper(x_train_story, y_train_story, x_test_story, y_test_story, 0.1)
    
# print results
print('F1: ', f1)
print('precision: ', precision)
print('recall: ', recall)

In [ ]:
# This is where the table needs data from for analysis
##### fits a regression model and assesses performance
f1, precision, recall, predictions_bugs = logistic_wrapper(x_train_bugs, y_train_bugs, x_test_bugs, y_test_bugs, 0.1)
    
# print results
print('F1: ', f1)
print('precision: ', precision)
print('recall: ', recall)

In [ ]:
##### grid search for regularization constant
# Grid of regularization constant values
alpha_list = [0.01, 0.05, 0.1, 0.5, 1, 2, 10, 20, 100]

# gets performance for each choice of regularization constant
output = [logistic_wrapper(x_train_train_overall, y_train_train_overall, x_train_valid_overall, y_train_valid_overall, alpha) for alpha in alpha_list]


### tabulates and prints performance by 
# reformats output into lists of metric values
output = list(zip(*output))
f1_alpha, precision_alpha, recall_alpha, predictions_alpha = [list(el) for el in output]

# adds row titles
alpha_list.insert(0,'alpha')
f1_alpha.insert(0,'F1')
precision_alpha.insert(0,'precision')
recall_alpha.insert(0,'recall')

# makes list of lists for tabulate.tabulate
table = [alpha_list, list(f1_alpha), precision_alpha, recall_alpha]

# prints table
print(tabulate(table))

In [ ]:
######## DEFINING WHICH SET TO FIND BEST ALPHA FOR ########
# Uncomment the set of training and testing data to find the best alpha value for
x_train = x_train_overall
y_train = y_train_overall
x_train_train = x_train_train_overall
x_train_valid = x_train_valid_overall
y_train_train = y_train_train_overall
y_train_valid = y_train_valid_overall

# x_train = x_train_controls
# y_train = y_train_controls
# x_train_train = x_train_train_controls
# x_train_valid = x_train_valid_controls
# y_train_train = y_train_train_controls
# y_train_valid = y_train_valid_controls

# x_train = x_train_graphics
# y_train = y_train_graphics
# x_train_train = x_train_train_graphics
# x_train_valid = x_train_valid_graphics
# y_train_train = y_train_train_graphics
# y_train_valid = y_train_valid_graphics

# x_train = x_train_story
# y_train = y_train_story
# x_train_train = x_train_train_story
# x_train_valid = x_train_valid_story
# y_ train_train = y_train_train_story
# y_train_valid = y_train_valid_story

# x_train = x_train_bugs
# y_train = y_train_bugs
# x_train_train = x_train_train_bugs
# x_train_valid = x_train_valid_bugs
# y_train_train = y_train_train_bugs
# y_train_valid = y_train_valid_bugs

In [ ]:
##### Looping the table through 30 random splits to find best alpha
# Array to keep track of average alpha Fscore
alpha_f1_average = [0, 0, 0, 0, 0, 0, 0, 0, 0]
# Loop for random splits
for i in range(1,30):
    x_train_train, x_train_valid, y_train_train, y_train_valid = train_test_split(x_train, y_train, test_size = 0.2)
    
    ##### grid search for regularization constant
    # Grid of regularization constant values
    alpha_list = [0.01, 0.05, 0.1, 0.5, 1, 2, 10, 20, 100]

    # gets performance for each choice of regularization constant
    output = [logistic_wrapper(x_train_train, y_train_train, x_train_valid, y_train_valid, alpha) for alpha in alpha_list]


    ### tabulates and prints performance by 
    # reformats output into lists of metric values
    output = list(zip(*output))
    f1_alpha, precision_alpha, recall_alpha, predictions_alpha = [list(el) for el in output]

    for count in range(0,9):
        alpha_f1_average[count] = alpha_f1_average[count] + f1_alpha[count]
    
    # adds row titles
    alpha_list.insert(0,'alpha')
    f1_alpha.insert(0,'F1')
    precision_alpha.insert(0,'precision')
    recall_alpha.insert(0,'recall')

    # makes list of lists for tabulate.tabulate
    table = [alpha_list, list(f1_alpha), precision_alpha, recall_alpha]
    
    # prints table
    # print(tabulate(table))
    
# print averages to compare manually
alpha_list = [0.01, 0.05, 0.1, 0.5, 1, 2, 10, 20, 100]

# print all averages
max_index = 0
for count in range(0,9):
    # print("Alpha Value: ", alpha_list[count],"Average F1 Score: ", alpha_f1_average[count]/30)
    if alpha_f1_average[count] > alpha_f1_average[max_index]:
        max_index = count
    
# print alpha with best average
print("Best Average Alpha: ", alpha_list[max_index], "\nAverage F1 Score: ", alpha_f1_average[max_index]/30)

In [ ]:
###### Choost prediction set ######
labels = y_train_overall
# labels = y_train_controls
# labels = y_train_graphics
# labels = y_train_story
# labels = y_train_bugs

predictions = predictions_overall
# predictions = predictions_controls
# predictions = predictions_graphics
# predictions = predictions_story
# predictions = predictions_bugs

In [ ]:
##### accessing examples for error analysis

# obtains list of false positive and negative indices
FPs = [i for i in range(len(predictions)) if predictions[i] == 1 and labels[i] == 0]
FNs = [i for i in range(len(predictions)) if predictions[i] == 0 and labels[i] == 1]

print(FPs)
print(FNs)

In [ ]:
### Testing a single review ###
# function that fits regression model and assesses performance
def testing_wrapper(training, x_fit, y_fit, review, alpha):
    
    ##### gets n-grams

    # sets ups n-gram calculator
    # character partition
    # ngram_calculator = CountVectorizer(ngram_range = (min_n, max_n), analyzer = 'char')
    # word partition
    ngram_calculator = CountVectorizer(ngram_range = (min_n, max_n), analyzer = 'word')
    ngram_calculator.fit(training['Comment Body'])
    # gets ngram (count) features
    x_review = ngram_calculator.transform([review])
    
    # runs logistic regression
    model = LogisticRegression(C = 1/alpha, solver = 'liblinear')
    model.fit(x_fit, y_fit)
    
    # calculates prediction
    predictions = model.predict(x_review)
        
    return predictions

In [ ]:
# Wrapper to generate predictions for a review
def prediction_wrapper(review):
    predictions_overall = testing_wrapper(train_overall, x_train_overall, y_train_overall, review, 0.1)
    predictions_controls = testing_wrapper(train_controls, x_train_controls, y_train_controls, review, 0.1)
    predictions_graphics = testing_wrapper(train_graphics, x_train_graphics, y_train_graphics, review, 0.1)
    predictions_story = testing_wrapper(train_story, x_train_story, y_train_story, review, 0.1)
    predictions_bugs = testing_wrapper(train_bugs, x_train_bugs, y_train_bugs, review, 0.1)
    print("Review: ", review)
    print("Overall Sentiment Prediction: ", predictions_overall)
    print("Controls Sentiment Prediction: ", predictions_controls)
    print("Graphics Sentiment Prediction: ", predictions_graphics)
    print("Story Sentiment Prediction: ", predictions_story)
    print("Bugs Sentiment Prediction: ", predictions_bugs)


In [ ]:
review = "This was a great game with bad controls"
prediction_wrapper(review)

review = "I did not enjoy this game. The graphics were awesome, but it was very buggy"
prediction_wrapper(review)